***
***
## Problema 2
Una refinería tiene dos aceites crudos disponibles, que tienen los rendimientos que se muestran en la siguiente tabla. Debido a las limitaciones del equipo y almacenamiento, la producción de gasolina, el queroseno y el aceite combustible debe ser limitada como también se muestra en esta tabla. No hay límites en la planta en la producción de otros productos tales como gasóleos.

El beneficio del procesamiento del crudo No. 1 es de 1.00/bbl y del crudo No. 2 es de 0.70/bbl.

Encontrar las velocidades de alimentación diaria óptima aproximados de los dos crudos a esta planta para maximizar el beneficio de producción.


|                     | Crudo 1 | Crudo 2 | Capacidad maxima de producción (bbl/día)  |
|---------------------|---------|---------|-------------------------------------------|
| Gasolina            | 70 %    | 31 %    | 6,000                                     |
| Queroseno           | 6 %     | 9 %     | 2,400                                     |
| Aceite  combustible | 24 %    | 60 %    | 12,000                                    |

In [2]:
using JuMP
using Ipopt

refineria = Model(solver=IpoptSolver())
@variable(refineria, c1 >=0)
@variable(refineria, c2 >=0)

@objective(refineria, Max, 5*c1 + 0.70*c2)

@constraints(refineria, begin
    0.70*c1 + 0.31*c2 <= 10000
    0.06*c1 + 0.09*c2 <= 2400
    0.24*c1 + 0.60*c2 <= 12000
    end)

solve(refineria)
println(refineria)
println("Costo maximizado = ", getobjectivevalue(refineria))
println("======================================")
println("Alimentación Crudo 1 = ", getvalue(c1), " \$ bbl/día")
println("Alimentación Crudo 2 = ", getvalue(c2), " \$ bbl/día")

This is Ipopt version 3.12.2, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:        0
Number of nonzeros in inequality constraint Jacobian.:        6
Number of nonzeros in Lagrangian Hessian.............:        0

Total number of variables............................:        2
                     variables with only lower bounds:        2
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:        0
Total number of inequality constraints...............:        3
        inequality constraints with only lower bounds:        0
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        3

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0 -

***
***
## Problema 3

Una alimentación a 3 unidades de procesamiento es divida en tres corrientes: $F_A$, $F_B$ y $F_C$. Dos productos son obtenidos: $P_1$ y $P_2$ (ver la siguiente Figura), y la produccion en % peso por unidad es: 

| Producto (% peso)   | Unidad A   | Unidad B   | Unidad C   |
|:-------------------:|:----------:|:----------:|:----------:|
| $P_1$               | 40         | 30         | 50         |
| $P_2$               | 60         | 70         | 50         |

Cada corriente tiene un valor en \$/bbl como se indica a continuación:

|    Corriente   | $F$  | $P_1$ | $P_2$ |
|:--------------:|:----:|:-----:|:-----:|
| Valor (\$/bbl) | 0.40 |  0.60 |  0.30 |

Debido a las limitaciones operacionales, existen ciertas limitaciones en los flujos de entrada:

1. La alimentación total de entrada no debe exceder 10,000 lb/día
2. La alimentación a cada unidad A, B y C no debe exceder 5,000 lb/día
3. No se puede usar mas de 4,000 lb/día de $P_1$ y no mas de 7,000 lb/día de $P_2$. 

En orden de determinar los valores de $F_A$, $F_B$ y $F_C$ que maximicen el beneficio diario, preparar el modelo matematico de este problema y su programación empleando JuMP.


![Figura 2](Fig/Problema2.png)

In [5]:
using JuMP
using Ipopt

m = Model(solver=IpoptSolver())

@variable(m, 0<= F <= 10000)
@variable(m, 0<= FA <= 5000)
@variable(m, 0<= FB <= 5000)
@variable(m, 0<= FC <= 5000)
@variable(m, 0<= P1 <= 4000)
@variable(m, 0<= P2 <= 7000)
    
@constraint(m, FA + FB + FC == F)
@constraint(m, FA*0.40 + FB *0.30 + FC*0.50 == P1)
@constraint(m, FA*0.60 + FB *0.70 + FC*0.50 == P2)

@objective(m, Max, 0.60*P1 + 0.30*P2 - 0.40*F)

println(m)
solve(m)

println("Costo maximizado = ", getobjectivevalue(m))
println("=========================================")
println("F = ", getvalue(F))
println("FA = ", getvalue(FA))
println("FB = ", getvalue(FB))
println("FC = ", getvalue(FC))
println("P1 = ", getvalue(P1))
println("P2 = ", getvalue(P2))

Max 0.6 P1 + 0.3 P2 - 0.4 F
Subject to
 FA + FB + FC - F == 0
 0.4 FA + 0.3 FB + 0.5 FC - P1 == 0
 0.6 FA + 0.7 FB + 0.5 FC - P2 == 0
 0 <= F <= 10000
 0 <= FA <= 5000
 0 <= FB <= 5000
 0 <= FC <= 5000
 0 <= P1 <= 4000
 0 <= P2 <= 7000

This is Ipopt version 3.12.2, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:       12
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:        0

Total number of variables............................:        6
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        6
                     variables with only upper bounds:        0
Total number of equality constraints.................:        3
Total number of inequality constraints...............:        0
        inequality constraints with only lo

***
***
## Problema 4

Una empresa de fabricación de productos químicos ha descontinuado la producción de una cierta linea de producción no rentable. Esto ha creado un considerable exceso de capacidad de producción en las tres existentes instalaciones de producción por lotes. La administración está considerando la posibilidad de dedicar este exceso de capacidad de uno o más de tres nuevos productos: Se pueden llamar producto 1, 2 y 3. La capacidad disponible en las unidades existentes tiene un limite que se resume en la siguiente tabla:

| Equipo | Tiempo disponible (h/semana) |
|:------:|:----------------------------:|
|    A   |         20                   |
|    B   |         10                   |
|    C   |         5                    |

Cada uno de los tres nuevos productos requiere el siguiente tiempo de procesamiento (h/batch):

| Equipo | Producto 1 | Producto 2 | Producto 3 |
|:------:|:----------:|:----------:|:----------:|
|    A   |     0.8    |     0.2    |     0.3    |
|    B   |     0.4    |     0.3    |     0.2    |
|    C   |     0.2    |     0.2    |     0.1    |

El departamento de ventas indica que las ventas potenciales para los productos 1 y 2 excede la velocidad maxima de produccion y que las ventas potenciales para el producto 3 es 20 lotes por semana. El beneficio economico por lotes es de \$ 20, \$ 6 y \$ 8, respectivamente, para los productos 1, 2 y 3. 

Formular un modelo de programación lineal para la determinación de cuanto de cada uno de los productos se deben producir para maximizar el beneficio economico. 

In [12]:
# Solución sin matrices
if isdefined(:JuMP) == false
    using JuMP
end

if isdefined(:Mosek) == false
    using Cbc
end

chem = Model(solver=CbcSolver())

@variable(chem, X1A >= 0)
@variable(chem, X1B >= 0)
@variable(chem, X1C >= 0)
@variable(chem, X2A >= 0)
@variable(chem, X2B >= 0)
@variable(chem, X2C >= 0)
@variable(chem, X3A >= 0)
@variable(chem, X3B >= 0)
@variable(chem, X3C >= 0)

@constraint(chem, X3A + X3B + X3C <= 20)
@constraint(chem, 0.8*X1A + 0.2*X2A + 0.3*X3A <= 20)
@constraint(chem, 0.4*X1B + 0.3*X2B + 0.2*X3B <= 10)
@constraint(chem, 0.2*X1C + 0.2*X2C + 0.1*X3C <=  5)
@objective(chem, Max, 20*sum(X1A + X1B + X1C) + 6*(X2A + X2B + X2C) + 8*(X3A + X3B + X3C))

status = solve(chem)

println(chem)
println("Costo maximizado = ", getobjectivevalue(chem))
println("======================================")
println("X1A = ", getvalue(X1A))
println("X2A = ", getvalue(X2A))
println("X3A = ", getvalue(X3A))
println("X1B = ", getvalue(X1B))
println("X2B = ", getvalue(X2B))
println("X3B = ", getvalue(X3B))
println("X1C = ", getvalue(X1C))
println("X2C = ", getvalue(X2C))
println("X3C = ", getvalue(X3C))

Max 20 X1A + 20 X1B + 20 X1C + 6 X2A + 6 X2B + 6 X2C + 8 X3A + 8 X3B + 8 X3C
Subject to
 X3A + X3B + X3C <= 20
 0.8 X1A + 0.2 X2A + 0.3 X3A <= 20
 0.4 X1B + 0.3 X2B + 0.2 X3B <= 10
 0.2 X1C + 0.2 X2C + 0.1 X3C <= 5
 X1A >= 0
 X1B >= 0
 X1C >= 0
 X2A >= 0
 X2B >= 0
 X2C >= 0
 X3A >= 0
 X3B >= 0
 X3C >= 0

Costo maximizado = 1600.0
X1A = 0.0
X2A = 100.0
X3A = 0.0
X1B = 25.0
X2B = 0.0
X3B = 0.0
X1C = 25.0
X2C = 0.0
X3C = 0.0


In [13]:
# Solución con matrices
if isdefined(:JuMP) == false
    using JuMP
end

if isdefined(:Mosek) == false
    using Cbc
end

chem = Model(solver=CbcSolver())

t=[0.8 0.2 0.3
    0.4 0.3 0.2
    0.2 0.2 0.1]

@variable(chem, X[1:3,1:3] >= 0)

@constraints(chem, begin
            sum(X[:,3]) <= 20
            sum(t[1,:].*X[1,:]) <= 20
            sum(t[2,:].*X[2,:]) <= 10
            sum(t[3,:].*X[3,:]) <=  5
            end)
    
@objective(chem, Max, 20*sum(X[:,1]) + 6*sum(X[:,2]) + 8*sum(X[:,3]))
solve(chem)

println(chem)
println("Costo maximizado = ", getobjectivevalue(chem))
println("======================================")
c = 1;
for i in ["A","B","C"]
    for j=1:3
        println("X$(j)$(i) = ", getvalue(X[c,j]))
    end
    c = c + 1;
end

Max 20 X[1,1] + 20 X[2,1] + 20 X[3,1] + 6 X[1,2] + 6 X[2,2] + 6 X[3,2] + 8 X[1,3] + 8 X[2,3] + 8 X[3,3]
Subject to
 X[1,3] + X[2,3] + X[3,3] <= 20
 0.8 X[1,1] + 0.2 X[1,2] + 0.3 X[1,3] <= 20
 0.4 X[2,1] + 0.3 X[2,2] + 0.2 X[2,3] <= 10
 0.2 X[3,1] + 0.2 X[3,2] + 0.1 X[3,3] <= 5
 X[i,j] >= 0 for all i in {1,2,3}, j in {1,2,3}

Costo maximizado = 1600.0
X1A = 0.0
X2A = 100.0
X3A = 0.0
X1B = 25.0
X2B = 0.0
X3B = 0.0
X1C = 25.0
X2C = 0.0
X3C = 0.0


***
***
## Problema 4

A chemical plant makes three products and uses three raw materials in limited supply as shown in Figure P7.23. Each of the three products is produced in a separate process (1,2,3) according to the schematic shown in the figure.

![Figura](Fig/P7.23.png)

The available A, B, and C do not have to be totally consumed.

![Tabla](Fig/Table38.png)

Set up the linear profit function and linear constraints to find the optimum product distribution.

In [14]:
using JuMP
using Ipopt

m = Model(solver=IpoptSolver())

@variable(m, A >=0)
@variable(m, B >=0)
@variable(m, C >=0)
@variable(m, E >=0)
@variable(m, F >=0)
@variable(m, G >=0)

@constraint(m, (2/3)*E + (2/3)*F + (1/2)*G == A)
@constraint(m, (1/3)*E + (1/3)*F + (1/6)*G == B)
@constraint(m, (1/6)*G == C)

@constraint(m, A <= 4000)
@constraint(m, B <= 3000)
@constraint(m, C <= 2500)

@objective(m, Max, 4*E + 3.3*F + 3.8*G - (1.5*A + 2*B + 2.5*C) - ((2/3)*E + (1/3)*F + G))

solve(m)

println("F(x) = ", getobjectivevalue(m))
println("A = ", getvalue(A))
println("B = ", getvalue(B))
println("C = ", getvalue(C))
println("E = ", getvalue(E))
println("F = ", getvalue(F))
println("G = ", getvalue(G))

This is Ipopt version 3.12.2, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:       10
Number of nonzeros in inequality constraint Jacobian.:        3
Number of nonzeros in Lagrangian Hessian.............:        0

Total number of variables............................:        6
                     variables with only lower bounds:        6
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Total number of equality constraints.................:        3
Total number of inequality constraints...............:        3
        inequality constraints with only lower bounds:        0
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        3

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0 -